# STAT 430 Unsupervised Learning Final Project
## Fashion MNIST Data Analysis
_Atharv Pathak & Jonas Reger_

Package Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.cm as cm
%matplotlib inline

import time
import math

from sklearn.cluster import KMeans, MiniBatchKMeans, Birch, DBSCAN, AgglomerativeClustering
from sklearn.manifold import TSNE
from sklearn.datasets import make_blobs, load_digits
from sklearn.metrics import adjusted_rand_score, silhouette_score
# , calinski_harabaz_score
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

from kmodes.kmodes import KModes

from scipy.spatial.distance import pdist, squareform
from scipy.cluster.hierarchy import linkage, dendrogram, cophenet

Fashion MNIST Sample Data Import

In [2]:
df = pd.read_csv('fashion_mnist_sample.csv', sep=',')
df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,Bag,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Trousers,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Dress,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Pullover,0,0,0,0,0,0,0,0,0,...,109,136,35,0,14,15,0,0,0,0
4,Ankle boot,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
df.describe()

,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
count,2000.0,2000.000000,2000.000000,2000.00000,2000.000000,2000.000000,2000.000000,2000.000000,2000.00000,2000.000000,...,2000.000000,2000.000000,2000.000000,2000.00000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000,2000.000000
mean,0.0,0.000500,0.016000,0.03400,0.132500,0.226000,0.515000,2.153500,4.94500,14.191500,...,33.232000,22.682500,16.067000,17.81600,23.696500,18.694500,9.060500,2.390500,0.619500,0.076500
std,0.0,0.022361,0.183741,0.22775,3.584602,4.371096,5.564823,13.976681,21.61237,37.662542,...,58.341795,48.890376,41.768994,44.33599,52.444853,46.832989,29.783391,15.563937,7.501657,1.870133
min,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.0,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,...,49.250000,5.000000,0.000000,0.00000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,0.0,1.000000,4.000000,3.00000,158.000000,181.000000,128.000000,190.000000,230.00000,238.000000,...,239.000000,250.000000,255.000000,241.00000,251.000000,255.000000,234.000000,208.000000,180.000000,62.000000


In [5]:
df.isnull().sum(axis = 0)

label       0
pixel1      0
pixel2      0
pixel3      0
pixel4      0
           ..
pixel780    0
pixel781    0
pixel782    0
pixel783    0
pixel784    0
Length: 785, dtype: int64